In [29]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
# Add classification_report to the list
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import xgboost as xgb
import shap
import warnings
warnings.filterwarnings('ignore')

# Load cleaned data and prepared splits (reuse from notebook 3)
df = pd.read_csv('amazon_sales_cleaned.csv')
# Recreate features and splits as done in notebook 3 (code omitted for brevity, assume we have X_train, X_test, etc.)

# For demonstration, we focus on two models: Random Forest Regressor for total_revenue and XGBoost Classifier for high_value.

# Ensure this logic is in Notebook 4 before the XGBoost section:
df['high_value'] = (df['total_revenue'] > 1000).astype(int)
feature_cols = ['price', 'discount_percent', 'quantity_sold', 'rating', 'review_count', 
                'year', 'month', 'day', 'day_of_week', 'cat_encoded', 'region_encoded', 'payment_encoded']

X = df[feature_cols]
y_hv = df['high_value']

from sklearn.model_selection import train_test_split
X_train_hv, X_test_hv, y_train_hv, y_test_hv = train_test_split(
    X, y_hv, test_size=0.2, random_state=42, stratify=y_hv
)


KeyError: "['cat_encoded', 'region_encoded', 'payment_encoded'] not in index"

In [30]:
# 1. Hyperparameter Tuning for Random Forest Regressor 

# Use RandomizedSearchCV for efficiency
param_dist_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)
rand_search_rf = RandomizedSearchCV(rf_base, param_dist_rf, n_iter=10, cv=3, scoring='r2', random_state=42, n_jobs=-1)
rand_search_rf.fit(X_train, y_train_rev)   # assuming X_train is original (unscaled) from notebook 3

print("Best RF params:", rand_search_rf.best_params_)
best_rf = rand_search_rf.best_estimator_

# Evaluate
pred_rf = best_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(y_test_rev, pred_rf))
r2_rf = r2_score(y_test_rev, pred_rf)
print(f"Tuned RF - RMSE: {rmse_rf:.2f}, R2: {r2_rf:.3f}")

Best RF params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Tuned RF - RMSE: 0.99, R2: 1.000


In [31]:
# 2. Hyperparameter Tuning for XGBoost Classifier (high-value) 

param_dist_xgb = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_clf = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
rand_search_xgb = RandomizedSearchCV(xgb_clf, param_dist_xgb, n_iter=10, cv=3, scoring='accuracy', random_state=42, n_jobs=-1)
rand_search_xgb.fit(X_train_hv, y_train_hv)   # assuming X_train_hv from notebook 3

print("\nBest XGBoost params:", rand_search_xgb.best_params_)
best_xgb = rand_search_xgb.best_estimator_

# Evaluate
pred_xgb = best_xgb.predict(X_test_hv)
acc_xgb = accuracy_score(y_test_hv, pred_xgb)
print(f"Tuned XGBoost - Accuracy: {acc_xgb:.3f}")
print("\nClassification Report:\n", classification_report(y_test_hv, pred_xgb))


Best XGBoost params: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1, 'colsample_bytree': 1.0}
Tuned XGBoost - Accuracy: 0.999

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      7629
           1       1.00      1.00      1.00      2371

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [32]:
# 3. Model Comparison Table 

# Collect scores from various models (including baseline from notebook 3)
results = {
    'Model': ['Linear Regression', 'Decision Tree', 'Random Forest', 'Gradient Boosting', 'XGBoost', 'Tuned RF'],
    'RMSE': [985.2, 876.4, 812.3, 789.5, 776.1, 764.8],  # example numbers, replace with actual
    'R2': [0.45, 0.56, 0.62, 0.64, 0.65, 0.67]
}
results_df = pd.DataFrame(results)
print("\nRegression Model Comparison:\n", results_df) 


Regression Model Comparison:
                Model   RMSE    R2
0  Linear Regression  985.2  0.45
1      Decision Tree  876.4  0.56
2      Random Forest  812.3  0.62
3  Gradient Boosting  789.5  0.64
4            XGBoost  776.1  0.65
5           Tuned RF  764.8  0.67


In [ ]:
# 4. Residual Diagnostics for Best Regressor 

residuals = y_test_rev - pred_rf
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.scatter(pred_rf, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Revenue')
plt.ylabel('Residuals')
plt.title('Residual Plot')

plt.subplot(1,2,2)
sns.histplot(residuals, bins=30, kde=True)
plt.xlabel('Residual')
plt.title('Residual Distribution')
plt.show()

# Check normality (Q-Q plot)
import scipy.stats as stats
plt.figure()
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot')
plt.show()
